In [1]:
import math
import os
import gc
import sys
import time

from functools import partial
from typing import Dict, List, Optional, Tuple, Union

from numba import jit, njit
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
BASE_DIR = '/home/dmitry/projects/dfdc'
SRC_DIR = os.path.join(BASE_DIR, 'src')
DATA_DIR = os.path.join(BASE_DIR, 'data/dfdc-videos')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2

import torch
import torchvision

from torch import Tensor
from torchvision import ops

import nvidia.dali as dali
from nvidia.dali.plugin.pytorch import DALIGenericIterator

# Pytorch_Retinaface
sys.path.insert(0, os.path.join(BASE_DIR, 'vendors/Pytorch_Retinaface'))
from data import cfg_mnet
from layers.functions.prior_box import PriorBox
from models.retinaface import RetinaFace
from utils.nms.py_cpu_nms import py_cpu_nms

# src
sys.path.insert(0, SRC_DIR)
from dataset.utils import read_labels
from detectors.retinaface import load_model, postproc_detections

In [4]:
from detectors.retinaface import init_detector, detect
from image import crop_square
from prepare_data import get_file_list, write_file_list
from video import VideoPipe

In [5]:
def show_image_bbox(img, boxes=None, figsize=(15,8)):
    plt.figure(figsize=figsize)
    if boxes is not None:
        for bbox in boxes:
            rect = plt.Rectangle(
                (bbox[0], bbox[1]), 
                bbox[2] - bbox[0],
                bbox[3] - bbox[1], fill=False,
                edgecolor='r', linewidth=1)
            if len(bbox) > 4:
                text = plt.text(
                    bbox[2]+10, bbox[1]-10, 
                    '%.02f' % bbox[4], color='r')
            plt.gca().add_patch(rect)
    plt.imshow(img)


def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [23]:
device = torch.device('cuda:1')
cfg = {**cfg_mnet, 'batch_size': 30}
weights = os.path.join(BASE_DIR, 'data/weights/mobilenet0.25_Final.pth')
detector = init_detector(cfg, weights, False).to(device)
detect_fn = partial(detect, model=detector, cfg=cfg, device=device)

Loading pretrained model from /home/dmitry/projects/dfdc/data/weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300


In [7]:
chunk_dirs = [f'dfdc_train_part_{i}' for i in range(9,10)]

In [8]:
df = read_labels(DATA_DIR, chunk_dirs=chunk_dirs)
print(df.shape)

(1736, 3)


In [9]:
files = get_file_list(df, 1555, 1900, DATA_DIR)
files

['/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/dsppvlvcom.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/xumrrieufz.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/zmgwqzewad.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/zqvbioegmr.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/koaythwkcn.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/vtmztrtrdy.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/veyarkmtry.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/potkccweye.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/cxfbhgmuyu.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/grielfiuwt.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/qazsmxyqru.mp4',
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/glgpfbebcx.mp4',
 '/home/dmitry/projects/dfdc

In [10]:
write_file_list(files, path='temp.txt')

pipe = VideoPipe('temp.txt', seq_len=30, stride=10, device_id=1, num_threads=1)
pipe.build()
num_samples = len(files)
num_samples_read = pipe.epoch_size('reader')

num_samples, num_samples_read

(181, 181)

In [18]:
pipe_out = pipe.run()
frames = pipe_out[0].as_cpu().as_array()
idx = pipe_out[1].as_cpu().as_array().item()
idx, files[idx], frames.shape

(6,
 '/home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_9/veyarkmtry.mp4',
 (1, 30, 2560, 1440, 3))

In [25]:
dets = detect_fn(frames[0])

RuntimeError: CUDA out of memory. Tried to allocate 1.65 GiB (GPU 1; 10.92 GiB total capacity; 3.71 GiB already allocated; 1.57 GiB free; 610.59 MiB cached)

In [24]:
dets = None

In [ ]:
f = 0

In [ ]:
frames.shape

In [ ]:
print(files[idx], f)
frame = frames[0,f]
boxes = dets[f][:,:4]
show_image_bbox(frame, boxes=boxes)

f += 1

In [ ]:
crop = crop_square(frame, boxes[0])

In [ ]:
%%time

num_frames = frames.shape[1]
faces = []

for i in range(num_frames):
    for det in dets[i]:
        face = crop_square(frames[0, i], det[:4])
        # face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
        faces.append(face)
            
# show_images(cropped_imgs, cols=4)